In [9]:
from ultralytics import YOLO
import os

# Path model
model_path = r"C:\Users\USER\Documents\a skripsi\riset\model_awal\best.pt"

# Path data.yaml
data_yaml = r"C:\Users\USER\Documents\a skripsi\workv1\self-training\3-iter\data.yaml"

# Output directory
project_dir = r"C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\test-1i"

# Load model
model = YOLO(model_path)

# Jalankan validasi
results = model.val(
    data=data_yaml,
    imgsz=640,
    conf=0.5,
    iou=0.5,
    # device=0,          # ganti ke 'cpu' jika tidak pakai GPU
    project=project_dir,
    name="val_iter3",  # subfolder output
    save_json=True,
    plots=True,
)

print("Validasi selesai.")
print(f"Hasil disimpan di: {os.path.join(project_dir, 'val_iter3')}")


Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 38.45.3 MB/s, size: 850.6 KB)
val: Scanning C:\Users\USER\Documents\a skripsi\workv1\self-training\3-iter\data_iter3 - Copy\labels... 2288 images, 348 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2288/2288 354.5it/s 6.5s0.1s
val: New cache created: C:\Users\USER\Documents\a skripsi\workv1\self-training\3-iter\data_iter3 - Copy\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 143/143 0.9it/s 2:311.1ss
                   all       2288       1940      0.988      0.979      0.989      0.941
Speed: 0.9ms preprocess, 43.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Saving C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\test-1i\val_iter3\predictions.json...
Results saved to C:\Users\US

In [8]:
import os
import shutil

# ===== PATH KONFIGURASI =====
SRC_IMAGES_DIR = r"C:\Users\USER\Documents\a skripsi\workv1\self-training\frames_iter"

DST_IMAGES_DIR = r"C:\Users\USER\Documents\a skripsi\workv1\self-training\3-iter\data_iter3 - Copy\images"
DST_LABELS_DIR = r"C:\Users\USER\Documents\a skripsi\workv1\self-training\3-iter\data_iter3 - Copy\labels"

# Ekstensi gambar yang dianggap valid
IMAGE_EXTENSIONS = (".jpg", ".jpeg", ".png")

# ===== PASTIKAN FOLDER TUJUAN ADA =====
os.makedirs(DST_IMAGES_DIR, exist_ok=True)
os.makedirs(DST_LABELS_DIR, exist_ok=True)

copied_count = 0
skipped_count = 0

for filename in os.listdir(SRC_IMAGES_DIR):
    if not filename.lower().endswith(IMAGE_EXTENSIONS):
        continue

    src_image_path = os.path.join(SRC_IMAGES_DIR, filename)
    dst_image_path = os.path.join(DST_IMAGES_DIR, filename)

    # Jika gambar sudah ada, abaikan
    if os.path.exists(dst_image_path):
        skipped_count += 1
        continue

    # Copy gambar
    shutil.copy2(src_image_path, dst_image_path)
    copied_count += 1

    # Buat file label kosong jika belum ada
    label_name = os.path.splitext(filename)[0] + ".txt"
    dst_label_path = os.path.join(DST_LABELS_DIR, label_name)

    if not os.path.exists(dst_label_path):
        open(dst_label_path, "w").close()

print("Proses selesai")
print(f"Gambar dicopy   : {copied_count}")
print(f"Gambar di-skip  : {skipped_count}")


Proses selesai
Gambar dicopy   : 348
Gambar di-skip  : 1940


In [2]:
# Epoch 5
import os
from ultralytics import YOLO
import csv

# =========================
# KONFIGURASI UTAMA
# =========================
# Path model
model_path = r"C:\Users\USER\Documents\a skripsi\riset\model_awal\best.pt"
# Path data.yaml generalisasi
DATA_YAML = r"C:/Users/USER/Documents/a skripsi/riset/data_eval_final/test generalisasi/data.yaml"

# Output folder utama
OUTPUT_DIR = "./test-1i"

ROOT_DIR = r"C:/Users/USER/Documents/a skripsi/riset/history/5-epoch-1020"

# Daftar model (10 path)
MODEL_PATHS = [
    model_path
]

CSV_OUT = os.path.join(OUTPUT_DIR, "results.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===================== HEADER CSV =====================
header = [
    "iteration",
    "TP", "FP", "TN", "FN",
    "precision", "recall", "f1_score",
    "mAP50", "mAP50_95"
]

rows = []

# ===================== LOOP VALIDASI =====================
for idx, model_path in enumerate(MODEL_PATHS, start=1):
    print(f"\nValidating model iterasi {idx}")
    model = YOLO(model_path)

    metrics = model.val(
        data=DATA_YAML,
        imgsz=640,
        conf=0.5,
        # iou=0.5,
        project=OUTPUT_DIR,
        name=f"iterasi_{idx}",
        save_json=False,
        plots=True
    )

    # Confusion Matrix
    cm = metrics.confusion_matrix.matrix
    # Format:
    # [[TP, FP],
    #  [FN, TN(background)]]

    TP = int(cm[0, 0])
    FP = int(cm[0, 1])
    FN = int(cm[1, 0])
    TN = 0  # Tidak bermakna untuk object detection
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

    # precision = float(metrics.box.precision[0])
    # recall = float(metrics.box.recall[0])
    # f1 = 2 * precision * recall / (precision + recall + 1e-9)
    precision = float(metrics.box.p[0])
    recall = float(metrics.box.r[0])
    f1 = float(metrics.box.f1[0])


    mAP50 = float(metrics.box.map50)
    mAP5095 = float(metrics.box.map)

    rows.append([
        idx,
        TP, FP, TN, FN,
        precision, recall, f1,
        mAP50, mAP5095
    ])

# ===================== SIMPAN CSV =====================
with open(CSV_OUT, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

print("\n✅ Validation selesai")
print(f"Hasil disimpan ke: {CSV_OUT}")



Validating model iterasi 1
Ultralytics 8.3.221  Python-3.11.9 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 569.8243.3 MB/s, size: 337.9 KB)
val: Scanning C:\Users\USER\Documents\a skripsi\riset\data_eval_final\test generalisasi\test\labels.cache... 522 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 522/522 257.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 0.6it/s 55.4s1.6ss
                   all        522        522          1       0.99      0.995      0.827
Speed: 1.0ms preprocess, 77.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to C:\Users\USER\Documents\a skripsi\workv1\app\notebook_penelitian\test-1i\iterasi_1
TP: 517, FP: 0, TN: 0, FN: 5

✅ Validation selesai
Hasil disimpan ke: ./test-1i\results.csv
